In [ ]:
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
%cd /mydrive/AdvancedML/

/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
/content/gdrive/My Drive/AdvancedML


In [ ]:
!pip install tweepy --upgrade
!pip install black[jupyter]

     |████████████████████████████████| 69 kB 3.1 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: tweepy
    Found existing installation: tweepy 3.10.0
    Uninstalling tweepy-3.10.0:
      Successfully uninstalled tweepy-3.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
     |████████████████████████████████| 1.4 MB 8.5 MB/s 
     |████████████████████████████████| 97 kB 6.1 MB/s 
     |████████████████████████████████| 843 kB 50.9 MB/s 
     |████████████████████████████

In [ ]:
# !black /mydrive/Colab\ Notebooks/AdvancedML.ipynb

In [ ]:
%env BEARER_TOKEN=AAAAAAAAAAAAAAAAAAAAANs%2BZQEAAAAApt1VDxcaN2l1Vp3%2Fi4Lw6GyvLuA%3DFOiqhrg523HTE9a82AwX2GZ8ldNO8Jl6H42bNUWoHTlcu84ySc

env: BEARER_TOKEN=AAAAAAAAAAAAAAAAAAAAANs%2BZQEAAAAApt1VDxcaN2l1Vp3%2Fi4Lw6GyvLuA%3DFOiqhrg523HTE9a82AwX2GZ8ldNO8Jl6H42bNUWoHTlcu84ySc


In [ ]:
import os, tweepy

BEARER_TOKEN = os.environ["BEARER_TOKEN"]

client = tweepy.Client(bearer_token=BEARER_TOKEN)

In [ ]:
def pre_process_text(text: str) -> str:
    text = text.replace("\n", "\\n").replace("\t", "\\t")
    return text


tweet_paginator = tweepy.Paginator(client.search_recent_tweets,
                              query="Chelsea FC -is:retweet lang:en",
                              tweet_fields=[
                                "created_at", 
                                "possibly_sensitive", 
                                "public_metrics",
                              ],
                              place_fields=[
                                "full_name",
                                "geo",
                                "country",
                                "country_code"    
                              ],
                              expansions="geo.place_id",
                              max_results=10,
                              limit=20)

In [ ]:
responses = []
for resp in tweet_paginator:
  responses.append(resp)

{}
{}
{'places': [<Place id=7f15dd80ac78ef40 full_name=Bristol, England>]}
{}
{}
{}
{'places': [<Place id=151b9e91272233d1 full_name=Liverpool, England>]}
{}
{}
{}
{}
{}
{}
{'places': [<Place id=53b67b1d1cc81a51 full_name=Birmingham, England>]}
{'places': [<Place id=151b9e91272233d1 full_name=Liverpool, England>]}
{}
{}
{'places': [<Place id=3554d7fe9a8821f9 full_name=Epsom, South East>]}
{}
{}


In [ ]:
found_places = {}

for resp in responses:
  places = resp.includes.get('places')
  if places:
    for place in places:
      found_places[place.id] = place

print(found_places)

{'7f15dd80ac78ef40': <Place id=7f15dd80ac78ef40 full_name=Bristol, England>, '151b9e91272233d1': <Place id=151b9e91272233d1 full_name=Liverpool, England>, '53b67b1d1cc81a51': <Place id=53b67b1d1cc81a51 full_name=Birmingham, England>, '3554d7fe9a8821f9': <Place id=3554d7fe9a8821f9 full_name=Epsom, South East>}


In [ ]:
for resp in responses:
  for tweet in resp.data:
    if tweet.geo:
      place_id = tweet.geo.get('place_id')
      coords = found_places[place_id].geo.get('bbox')
      x = (coords[0]+coords[2])/2
      y = (coords[1]+coords[3])/2
      print(
          found_places[place_id].full_name,
          f"{x},{y}",
          tweet.created_at, 
          tweet.possibly_sensitive, 
          tweet.public_metrics.get('retweet_count'), 
          tweet.public_metrics.get('reply_count'),
          tweet.public_metrics.get('like_count'),
          tweet.public_metrics.get('quote_count'),
          pre_process_text(tweet.text),
      )
    else:
      print(
        None,
        None,
        tweet.created_at,
        tweet.possibly_sensitive,
        tweet.public_metrics.get('retweet_count'), 
        tweet.public_metrics.get('reply_count'),
        tweet.public_metrics.get('like_count'),
        tweet.public_metrics.get('quote_count'),
        pre_process_text(tweet.text),
      )

None None 2022-03-10 22:55:01+00:00 False 0 0 0 0 @NizaarKinsella It’s not in support of Romans dealings outside of Chelsea FC. It’s a Thank You for what he’s done FOR Chelsea FC. There’s a HUGE difference.
None None 2022-03-10 22:54:34+00:00 False 0 0 0 0 Yikes @ Chelsea Fc
None None 2022-03-10 22:53:57+00:00 False 0 0 0 0 @Quack1876 @Boro Only cos we are playing them in the cup, ONLY cos we get a bye, let's remember we are footy fans and it not Chelsea Fc fault they are owned by  DH.
None None 2022-03-10 22:53:33+00:00 False 0 0 1 0 @JayVtid @FabrizioRomano This is why Gerrad is the best Prem Manager and Aston Villa is clear of those oil fc Chelsea and City.
None None 2022-03-10 22:53:07+00:00 False 0 0 0 0 @ChelseaFC I hope Thomas Tuchel stays with us into the new era of Chelsea FC. He is easily the best manager we have had in a long time.
None None 2022-03-10 22:53:05+00:00 False 0 0 0 0 @SkySportsPL @KerouacNath Chelsea FC fans got the blues!\n#ChelseaFC #RomanAbramovic
None None 

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request

# Tasks:
# emoji, emotion, hate, irony, offensive, sentiment
# stance/abortion, stance/atheism, stance/climate, stance/feminist, stance/hillary

task = "sentiment"
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"

tokenizer = AutoTokenizer.from_pretrained(MODEL)

# download label mapping
labels = []
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode("utf-8").split("\n")
    csvreader = csv.reader(html, delimiter="\t")
labels = [row[1] for row in csvreader if len(row) > 1]

# TF
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)

Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [ ]:
text = "I love crackers"
encoded_input = tokenizer(text, return_tensors="tf")
output = model(encoded_input)
scores = output[0][0].numpy()
scores = softmax(scores)

ranking = np.argsort(scores)
ranking = ranking[::-1]
for i in range(scores.shape[0]):
    l = labels[ranking[i]]
    s = scores[ranking[i]]
    print(f"{i+1}) {l} {np.round(float(s), 4)}")

1) positive 0.9777
2) neutral 0.0181
3) negative 0.0042


In [ ]:
import csv

with open("innovators.csv", "w", newline="") as file:
    writer = csv.writer(file, delimiter=",", quoting=csv.QUOTE_ALL)
    writer.writerow(["Text", "Date", "Sentiment"])

In [ ]:
%ls

cardiffnlp/  innovators.csv
